In [1]:
import cv2
from typing import List
import numpy as np
import glob
import os 

In [2]:
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

# TODO Complete the method, use every argument
def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def write_image(output_folder: str, img_name: str, img: np.array):
    # Crea el camino completo para la imagen
    img_path = os.path.join(output_folder, img_name)
    # Guarda la imagen en la ruta especificada
    cv2.imwrite(img_path, img)


In [3]:
imgs_path = glob.glob("../images/raw_images/*.png")
imgs = load_images(imgs_path)

In [4]:
imgs_path = glob.glob("../images/test_images/*.png")
test_imgs = load_images(imgs_path)

In [218]:
print(imgs)

[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [  0,   0,   0],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [  0,   0,   0],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [  0,   0,   0],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [  0,   0,   0],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [  0,   0,   0],
        [255, 255, 255]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]

Segmentación en color rojo para detectar el numero en la imagen

In [6]:


def color_segment(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Definir el rango de verde claro en HSV
    # Aproximación: H para verde claro ~ 40-80, S alta (por ejemplo, 100-255) y V moderado a alto (50-255)
    lower_green = np.array([40, 30, 20])   # Rango inferior para verde muy oscuro (S y V más bajos)
    upper_green = np.array([80, 255, 255])  # Rango superior para verde claro a verde más oscuro
    
    
    # Crear la máscara para el verde claro
    green_mask = cv2.inRange(hsv_img, lower_green, upper_green)
    
    # Aplicar la máscara para obtener solo el segmento verde
    green_segmented = cv2.bitwise_and(img, img, mask=green_mask)

     # Definir un kernel para la dilatación (puede ser rectangular, elíptico, o cruzado)
    kernel = np.ones((7, 7), np.uint8)  # Un kernel de 5x5 de unos (puedes ajustarlo)
    
    # Aplicar dilatación para unir las áreas cercanas
    dilated_mask = cv2.dilate(green_mask, kernel, iterations=1)  # Puedes ajustar el número de iteraciones
    
    # Aplicar la máscara dilatada para obtener solo el segmento verde
    green_segmented = cv2.bitwise_and(img, img, mask=dilated_mask)

    return dilated_mask, green_segmented


# Ejemplo de uso con una imagen
img = imgs[1]  # Suponiendo que 'imgs' es una lista de imágenes

# Obtener la máscara final y la imagen segmentada
green_mask, green_segmented = color_segment(img)

# Mostrar las imágenes
show_image(img, "Imagen Original")
show_image(green_mask, "Máscara Verde")
show_image(green_segmented, "Imagen Segmentada - Verde Claro")


Detección de contornos para quedarnos solo con el número que aparece en la imágen

In [7]:
def extract_number_from_image(green_mask: np.array, img: np.array) -> np.array:
    # Encontrar los contornos en la máscara roja
    contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Encontrar el contorno más grande, que debería corresponder al número
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Obtener la caja delimitadora (bounding box) del contorno más grande
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Recortar la imagen usando la caja delimitadora
        cropped_img = img[y:y+h, x:x+w]
        return cropped_img
    else:
        # Si no se encuentra un contorno, devolver la imagen original
        return img

# Recortar la parte de la imagen donde hay más rojo
cropped_green_area = extract_number_from_image(green_mask, img)
resized_image = cv2.resize(cropped_green_area, (200,264), interpolation=cv2.INTER_LINEAR)

show_image(resized_image, "Región Recortada y redimensionada con 'Más Verde'")


Guardamos todas las cropped images redimensionadas, servirá para comparar y hacer la clasificación después

In [221]:
imgs_path = glob.glob("../images/raw_images/*.png")
print(imgs_path)

['../images/raw_images\\0.png', '../images/raw_images\\1.png', '../images/raw_images\\2.png', '../images/raw_images\\3.png', '../images/raw_images\\4.png', '../images/raw_images\\5.png', '../images/raw_images\\6.png', '../images/raw_images\\7.png', '../images/raw_images\\8.png', '../images/raw_images\\9.png']


In [8]:
j = 0 
for img in imgs:
    green_mask,green_segmented = color_segment(img)
    cropped_green_area = extract_number_from_image(green_mask, img)
    resized_image = cv2.resize(cropped_green_area, (200,264), interpolation=cv2.INTER_LINEAR)
    write_image(f'../images/cropped_images/',str(j)+'.png', resized_image )
    j = j+1

In [9]:
def detect_number(cropped_resized_img):
    '''Función para detectar número comparando los templates de las fotos'''
    digit_templates = [cv2.imread(f'../images/cropped_images/{i}.png', cv2.IMREAD_GRAYSCALE) for i in range(10)]
    cropped_resized_img = np.uint8(cropped_resized_img)
    digit_templates = [np.uint8(template) for template in digit_templates]
    cropped_resized_img = cv2.cvtColor(cropped_resized_img, cv2.COLOR_BGR2GRAY)
    best_match = None
    best_match_score = -1
    for i, template in enumerate(digit_templates):
        res = cv2.matchTemplate(cropped_resized_img, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        # Si encontramos una coincidencia mejor que la anterior
        if max_val > best_match_score:
            best_match_score = max_val
            best_match = i
    return best_match
        

In [16]:
img = cv2.imread(f'../images/test_images/face_test.jpg')
img = cv2.resize(img, (200,264), interpolation=cv2.INTER_LINEAR)
green_mask,green_segmented = color_segment(img)
show_image(green_mask)
cropped_green_area = extract_number_from_image(green_mask, img)
resized_image = cv2.resize(cropped_green_area, (200,264), interpolation=cv2.INTER_LINEAR)
show_image(resized_image)
digit_templates = [cv2.imread(f'../images/cropped_images/{i}.png', cv2.IMREAD_GRAYSCALE) for i in range(10)]
detected = detect_number(resized_image)
print(detected)

6
